In [5]:
import ollama


model = "llama3:8b"

ollama.pull(model)

{'status': 'success'}

In [6]:
ollama.generate(model=model, prompt="answer \"Test\"")

{'model': 'llama3:8b',
 'created_at': '2024-05-18T09:36:04.1690664Z',
 'response': "You want to test me, huh? Well, I'm ready! My answer is... TEST!",
 'done': True,
 'context': [128006,
  882,
  128007,
  198,
  198,
  9399,
  220,
  1,
  2323,
  1,
  128009,
  128006,
  78191,
  128007,
  198,
  198,
  2675,
  1390,
  311,
  1296,
  757,
  11,
  57843,
  30,
  8489,
  11,
  358,
  6,
  76,
  5644,
  0,
  3092,
  4320,
  374,
  1131,
  13916,
  0,
  128009],
 'total_duration': 49124619500,
 'load_duration': 14799307200,
 'prompt_eval_count': 15,
 'prompt_eval_duration': 10801722000,
 'eval_count': 21,
 'eval_duration': 23507062000}

In [ ]:
# TODO: load Testset

In [ ]:
# TODO: Define Prompt function

In [ ]:
# TODO: generate responses in appropriate format

In [ ]:
# TODO: Trigger evaluation